## Small script to create the Supplementary Figure of the inter/intra- per center


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statannot import add_stat_annotation
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
import itertools 

# ignore warnings for easier plotting
import warnings
warnings.filterwarnings("ignore")

from load_data import load_data

In [3]:
sns.set(style='darkgrid', context='talk', palette='Set1', font="serif")

In [ ]:
# Hue colors order and names
# auxiliar dict to put correct order in the figure
map_hue_color = {
    "EDSS_group": ["EDSS<3", "EDSS>=3"],
    "SDMT_group": ["SDMT>=40", "SDMT<40"],
    "G_norm": ['G_z<=0', 'G_z>0'], 
    "Corr_norm": ['C_z<=0', 'C_z>0'],
    "disease": ["HC", "MS"],
    "GROUP": ["HC", "CIS", "RRMS", "SPMS", "PPMS"],
    "CENTER": ["CLINIC", "MAINZ", "MILAN", "NAPLES", "OSLO", "LONDON", "AMSTERDAM"],
    "groups" : ["HC", "EDSS<3", "EDSS>=3", "SDMT>=40", "SDMT<40"]
    # "GROUP": ["HC", "RRMS", "SPMS", "PPMS"]
}

# get the progressives together.
mapping_prog = {
    "HC": "HC",
    "CIS": "CIS", 
    "RRMS": "RRMS", 
    "SPMS": "PMS", 
    "PPMS": "PMS"
}

df_merged = load_data()
print(df_merged.columns.values)

In [ ]:
print(len(df_merged))
x1 = "Comm_ratio"
x2 = "CC_ratio_area"

y1 = "SC_spl_full"
y2 = "SC_eff_full"


tidy = (
    df_merged[["Comm_ratio", "CC_ratio_area", "SC_spl_full", "SC_eff_full"]].stack() # pull the columns into row variables   
      .to_frame() # convert the resulting Series to a DataFrame
      .reset_index() # pull the resulting MultiIndex into the columns
      .rename(columns={0: 'val'}) # rename the unnamed column
)

print(tidy.head())


In [34]:

def bonferroni_correction(p_value, m, alpha=0.05):
    """
    Perform Bonferroni correction on a list of p-values.
    
    Parameters:
    - p_values: list or numpy array of p-values
    - alpha: significance level
    
    Returns:
    - corrected_p_values: list of Bonferroni corrected p-values
    - reject: boolean array indicating which null hypotheses are rejected
    """
    corrected_p_value = np.minimum(1, p_value * m)  # Ensure p-values don't exceed 1
    reject = corrected_p_value < alpha
    return corrected_p_value

# annotate function
def annotate(col1, col2, data, multihue=False, **kwargs):
    if len(data[col1]) >= 2 or len(data[col2]) >= 2:
        r, p = stats.pearsonr(data[col1], data[col2])
        p = bonferroni_correction(p, 28) # Total number of comparisons

        ax = plt.gca()

        # hardcoded, udpate if working with other labels
        x_pos = 0.05
        if multihue:
            label = kwargs["label"]
            dict_of_labels = {"CIS": 0, "RRMS": 1, "PMS": 2, 'MS':0, 'HC':1}
            y_pos = 0.9 - dict_of_labels[kwargs["label"]]*0.07
        else:
            label =  " "
            y_pos = 0.9
        
        ax.text(x_pos, y_pos, f'{label}: ' + 'r={:.2f}, p={:.2g}'.format(r, p),
                transform=ax.transAxes)


In [ ]:
sns.set(style="darkgrid", context="talk", palette="tab10", font="serif")

x = "Comm_ratio"

y1 = "SC_spl_full"
y2 = "SC_eff_full"

axis_labels = [("Commisural ratio", "Average SPL"), ("Commisural ratio", "Efficiency")]


for y, labels in zip([y1, y2], axis_labels):

    results = smf.ols(f'{x} ~ AGE + C(SEX)', data=df_merged).fit()
    df_merged[f"{x}_C"] = results.resid

    results = smf.ols(f'{y} ~ AGE + C(SEX)', data=df_merged).fit()
    df_merged[f"{y}_C"] = results.resid

    df_merged['disease'] = np.where((df_merged['GROUP']== 'HC'), "HC", "MS")
    df_merged_hc = df_merged[df_merged.disease == "HC"]
    g = sns.lmplot(data=df_merged, x=f"{x}_C", y=f"{y}_C", scatter_kws={'s':25}, col="CENTER", hue="disease", sharey=True, sharex=True)
    g.map_dataframe(annotate, f"{x}_C", f"{y}_C", multihue=True)
    g.set_axis_labels(labels[0], labels[1])

    g.set_yticklabels